In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import numpy as np
import copy

carico il file dei commenti che ho estrapolato tramite il mio sondaggio per cercare di analizzare il tutto e capire che cosa ho ottenuto

## Controllo validità dati iniziale

In [2]:
df = pd.read_csv(
    "./responses_27mag_FULL.csv",
    sep=";",
    quoting=csv.QUOTE_MINIMAL,
    encoding="utf-8",
    engine="python"
)

Mi sono accorta che nel deploy ho messo poi le label sbagliate, quindi le vado a sistemare tutte nel file nuovo df

In [3]:
df['age_group'].replace('18-25', '18-29', inplace=True)
df['age_group'].replace('26-35', '30-45', inplace=True)
df['age_group'].replace('36-45', '46-60', inplace=True)
df['age_group'].replace('46', '61+', inplace=True)

In [4]:
age_group_counts = df['age_group'].value_counts()
gender_counts = df['gender'].value_counts()

age_group_counts, gender_counts

(18-29    106
 30-45     81
 46-60     22
 61+        2
 Name: age_group, dtype: int64,
 Female    115
 Male       96
 Name: gender, dtype: int64)

## PRE-PREPROCESSING

### 1. one-hot encoding variabili categoriche

In [5]:
# Mappiamo i livelli di tossicità a numeri
tossicita_map = {
    "Non toxic": 0,
    "A little toxic": 1,
    "Toxic": 2,
    "Extremely toxic": 3,
    "Non racist": 0,
    "A little racist": 1,
    "Racist": 2,
    "Extremely racist": 3,
    "Non harassing": 0,
    "A little harassing": 1,
    "Harassing": 2,
    "Extremely harassing": 3,
    "Non vulgar": 0,
    "A little vulgar": 1,
    "Vulgar": 2,
    "Extremely vulgar": 3,
    "Non violent": 0,
    "A little violent": 1,
    "Violent": 2,
    "Extremely violent": 3
}

In [6]:
# Mappiamo la tipologia di tossicità
# 0: Toxicity
# 1: Racism
# 2: Harassment
# 3: Vulgarity      
# 4: Violence

type_toss_map = {
    "Non toxic": 0, 
    "A little toxic": 0,
    "Toxic": 0,
    "Extremely toxic": 0,

    "Non racist": 1,
    "A little racist": 1,
    "Racist": 1,
    "Extremely racist": 1,

    "Non harassing": 2,
    "A little harassing": 2,
    "Harassing": 2,
    "Extremely harassing": 2,

    "Non vulgar": 3,
    "A little vulgar": 3,
    "Vulgar": 3,
    "Extremely vulgar": 3,

    "Non violent": 4,
    "A little violent": 4,
    "Violent": 4,
    "Extremely violent": 4
}

In [7]:
# Codifica gender
gender_map = {
    'Male': 0, 
    'Female': 1}
df['gender'] = df['gender'].map(gender_map)

In [8]:
# Codifica age_group
age_map = {
    '18-29': 1,
    '30-45': 2, 
    '46-60': 3, 
    '61+': 4}
df['age_group'] = df['age_group'].map(age_map)

In [9]:
 # Escludi le prime 3 colonne: 'id', 'age_group', 'gender'
domande_colonne = [col for col in df.columns if 'q' in col and 'text' in col]
risposte_colonne = [col for col in df.columns if 'q' in col and 'response' in col]

In [10]:
data = []
for i, row in df.iterrows():
    for j in range(len(domande_colonne)):  # Per ogni domanda
        domanda = row[domande_colonne[j]]
        risposta = row[risposte_colonne[j]]
        tossicita = tossicita_map.get(risposta, -1)  # Mappa la risposta alla tossicità
        data.append({
            'commento': domanda,
            'valutazione': risposta,
            'tossicita': tossicita,
            'tipo_tossicita': type_toss_map.get(risposta, -1),  # Mappa la risposta al tipo di tossicità
            'age_group': row['age_group'],
            'gender': row['gender'],
            'utente_id': row['id'] if 'id' in row else i  # se esiste un ID, altrimenti usa i
        })

df = pd.DataFrame(data)

Va bene perchè sono 25 commenti da valutare, quindi 211*25 = 5275 commenti in tutto (anche ripetuti)


### 2. conteggio unicità commenti e tossicità

In [11]:
# Conta le occorrenze di ciascuna valutazione 
conta_risposte = df['valutazione'].value_counts()
conta_risposte

Non toxic              918
Extremely violent      647
Racist                 472
A little harassing     469
A little vulgar        389
Harassing              376
A little racist        366
Vulgar                 354
Violent                286
Non vulgar             241
Non harassing          153
Extremely racist       109
A little violent       109
Non racist              95
A little toxic          85
Extremely vulgar        80
Toxic                   47
Extremely harassing     45
Non violent             25
Extremely toxic          9
Name: valutazione, dtype: int64

In [12]:
# Conta le occorrenze di ciascun commento
conta_domanda = df['commento'].value_counts()
conta_domanda

Death to capitalism                                                                                                                                                                                                 33
Traps are gay.                                                                                                                                                                                                      30
Nazi punk fuck off                                                                                                                                                                                                  15
Kill them. Kill them now                                                                                                                                                                                            14
Coffee oatmeal with low cal sweetner is the shit                                                                                            

Possiamo quindi dire che sono stati annotati in maniera differente 979 diversi commenti (commenti unici)

In [13]:
# seleziono i commenti che compiaiono almeno 3 volte
domande_min3 = conta_domanda[conta_domanda >= 3]
domande_min3

Death to capitalism                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             33
Traps are gay.                                                                                                                                                                                                                                                                                                                                                                                                       

Su un totale di 979 commenti, ci sono 890 commenti che sono stati valutati almeno 3 volte --> possiamo fare la media della loro valutazione per estrarne la tossicità

In [14]:
# adesso mi seleziono i commenti che sono comparsi un numero minore di 3 volte
commenti_non_3 = conta_domanda[conta_domanda < 3].index.tolist()
print(len(commenti_non_3))

89


mi torna che ci siano 89 commenti che non sono stati presentati almeno 3 volte: 89 (minore 3) + 890 (maggiore di 3) = 979 (totale)

!! con gli 89 commenti che sono stati valutati <3 volto, posso pensare di fare un secondo round di valutazione così di ottinere un numero di risposte sufficienti anche per questi commenti ed utilizzarli all'interno dei vari modelli di regressione che andrò a creare.

# PREPROCESSING DEI COMMENTI

In [15]:
# mi faccio una copia per non modificare il df creato 
df_copy = copy.deepcopy(df)

In [16]:
df_copy['commento'].describe()

count                    5275
unique                    979
top       Death to capitalism
freq                       33
Name: commento, dtype: object

### EMOJI ANALISI

In [17]:
import emoji
import emojis

#https://emojis.readthedocs.io/en/latest/index.html

In [18]:
# mi faccio una copia per non modificare il df creato 
df_emoji = copy.deepcopy(df_copy)
df_emoji.drop(columns=['utente_id', 'valutazione', 'age_group', 'gender', 'tossicita', 'tipo_tossicita'], inplace=True)
df_emoji

,commento
0,is this a satire subreddit?
1,Dayum.
2,And they wouldn’t be scrubbing the floors if t...
3,Good point. I mean does Nebraska really exist?...
4,or a yuumi/nami/lulu/zilean/ any enhancer play...
...,...
5270,Commies are going to commie. These people hate...
5271,Yeah but then the good guys win and rape you t...
5272,If you're white and use your gun to defend you...
5273,I will rape Elon Musk


In [19]:
# estraggo un singolo commento 
commento_emoji = df_emoji.iloc[562]
commento_emoji

commento    No one called me on my bullshit.  My comment i...
Name: 562, dtype: object

In [20]:
# Define the function to extract emoji information (number, set and list of emojis)
def estrazione_emoji(listLine):
    for i in listLine:
        count_e = emoji.emoji_count(i["text"])
        #if count_e != 0:
         #   print(count_e)
        set_e = ''.join(emoji.distinct_emoji_list(i["text"]))
        data_list_e = emoji.emoji_list(i["text"])
        if data_list_e:
            str_e = ''.join([entry['emoji'] for entry in data_list_e])
        else:
            str_e = ''

        standard_s = emojis.decode(i["text"]) # the emojis are decoded (e.g. the smiley face becomes :smile:)
        #standard_s.replace(':','') #used to remove the colons
        i["text"] = standard_s
        i["emoji_count"] = count_e
        i["emoji_unique"] = set_e
        i["emoji_list"] = str_e

In [21]:
# 1. Applico decode() per convertire le emoji in :emoji_testo
df_emoji["emoji_decoded"] = df_emoji["commento"].apply(lambda x: emojis.decode(x))
# elimino i :
df_emoji["emoji_decoded"] = df_emoji["emoji_decoded"].str.replace(":", "", regex=False)

# 2. Applico get() per estrarre un set di emoji
df_emoji["emoji_set"] = df_emoji["commento"].apply(lambda x: emojis.get(x))

# 3. Applico count() per contare quante emoji ci sono nel commento
df_emoji["emoji_count"] = df_emoji["commento"].apply(lambda x: emojis.count(x))

df_emoji

,commento,emoji_decoded,emoji_set,emoji_count
0,is this a satire subreddit?,is this a satire subreddit?,{},0
1,Dayum.,Dayum.,{},0
2,And they wouldn’t be scrubbing the floors if t...,And they wouldn’t be scrubbing the floors if t...,{},0
3,Good point. I mean does Nebraska really exist?...,Good point. I mean does Nebraska really exist?...,{},0
4,or a yuumi/nami/lulu/zilean/ any enhancer play...,or a yuumi/nami/lulu/zilean/ any enhancer play...,{},0
...,...,...,...,...
5270,Commies are going to commie. These people hate...,Commies are going to commie. These people hate...,{},0
5271,Yeah but then the good guys win and rape you t...,Yeah but then the good guys win and rape you t...,{},0
5272,If you're white and use your gun to defend you...,If you're white and use your gun to defend you...,{},0
5273,I will rape Elon Musk,I will rape Elon Musk,{},0


In [22]:
df_emoji['emoji_set'].value_counts()

{}                       5191
{😂}                        21
{😎}                        12
{🙄}                         8
{🇨🇳}                        7
{🤡, 😭}                      6
{😅}                         6
{♂️, 🤔, 🤷}                  5
{🤣}                         5
{💀}                         4
{🤬}                         4
{🤣, 😎, 😆, 🤑, 😂, 🤡, 😏}       4
{🤨}                         2
Name: emoji_set, dtype: int64

Ho preso il df originale a cui ho tolto tutte le colonne in modo da avere solo quella che mi interessa del commmento. A questo punto, ho chiamato la libreria EMOJIS che per ogni commento mi ha contato le emoji singole, salvato il set di emoji e decodificato il commento in una nuova riga. 

### PAROLE IN MAIUSCOLO

- Se l'intera frase è tutta maiuscola → prendi tutte le parole >= 2 caratteri, escludendo I.
- Se non è tutta maiuscola → prendi solo parole tutte maiuscole che rispettano le regole.
- Non prendere parole capitalizzate (es. Facebook se non è tutta maiuscola).

In [23]:
def trova_maiuscole(testo):
    if not isinstance(testo, str):
        return []
    
    parole = testo.split()
    
    # Caso 1: frase tutta maiuscola, allora prendi tutte le parole >= 2 caratteri (escludendo 'I')
    if testo.isupper():
        return [p for p in parole if len(p) >= 2 and p != "I"]
    
    # Caso 2: frase NON tutta maiuscola, allora prendi solo parole TUTTE maiuscole, >= 2 caratteri, escludendo 'I'
    return [p for p in parole if p.isupper() and len(p) >= 2 and p != "I"]

In [24]:
df_emoji["parole_maiusc"] = df_emoji["emoji_decoded"].apply(trova_maiuscole)
df_emoji

,commento,emoji_decoded,emoji_set,emoji_count,parole_maiusc
0,is this a satire subreddit?,is this a satire subreddit?,{},0,[]
1,Dayum.,Dayum.,{},0,[]
2,And they wouldn’t be scrubbing the floors if t...,And they wouldn’t be scrubbing the floors if t...,{},0,[]
3,Good point. I mean does Nebraska really exist?...,Good point. I mean does Nebraska really exist?...,{},0,[]
4,or a yuumi/nami/lulu/zilean/ any enhancer play...,or a yuumi/nami/lulu/zilean/ any enhancer play...,{},0,[]
...,...,...,...,...,...
5270,Commies are going to commie. These people hate...,Commies are going to commie. These people hate...,{},0,[]
5271,Yeah but then the good guys win and rape you t...,Yeah but then the good guys win and rape you t...,{},0,[]
5272,If you're white and use your gun to defend you...,If you're white and use your gun to defend you...,{},0,[]
5273,I will rape Elon Musk,I will rape Elon Musk,{},0,[]


### METTERE TUTTO IN MINUSCOLO

In [25]:
# Ora porto il testo in minuscolo
df_emoji["emoji_decoded"] = df_emoji["emoji_decoded"].str.lower()
df_emoji

,commento,emoji_decoded,emoji_set,emoji_count,parole_maiusc
0,is this a satire subreddit?,is this a satire subreddit?,{},0,[]
1,Dayum.,dayum.,{},0,[]
2,And they wouldn’t be scrubbing the floors if t...,and they wouldn’t be scrubbing the floors if t...,{},0,[]
3,Good point. I mean does Nebraska really exist?...,good point. i mean does nebraska really exist?...,{},0,[]
4,or a yuumi/nami/lulu/zilean/ any enhancer play...,or a yuumi/nami/lulu/zilean/ any enhancer play...,{},0,[]
...,...,...,...,...,...
5270,Commies are going to commie. These people hate...,commies are going to commie. these people hate...,{},0,[]
5271,Yeah but then the good guys win and rape you t...,yeah but then the good guys win and rape you t...,{},0,[]
5272,If you're white and use your gun to defend you...,if you're white and use your gun to defend you...,{},0,[]
5273,I will rape Elon Musk,i will rape elon musk,{},0,[]


### TOGLIERE CONTRAZIONI E PUNTEGGIATURA

In [26]:
# https://pypi.org/project/contractions/
# !pip install contractions
# !pip install num2words
# !pip install nltk

from html import unescape
import re
import string
import contractions
from num2words import num2words
from nltk.corpus import stopwords


In [27]:
def espandi_contrazioni(testo):
    if isinstance(testo, str):
        return contractions.fix(testo, slang=True)
    return testo

df_emoji["emoji_decoded"] = df_emoji["emoji_decoded"].apply(espandi_contrazioni)

In [28]:
punteggiatura = ['"', '$', '%', '&', "'", '(', ')', '*', '+', ',',
            '/', ':', ';', '<', '=', '>', '@', '[', '\\', ']', '^', '_',
            '`', '{', '|', '}', '~', '»', '«', '“', '”', '#', '!', '?', '.', ':']
punct_pattern = re.compile("[" + re.escape("".join(punteggiatura)) + "]")

stop_words = set(stopwords.words("english"))

def text_cleaning(testo):
    if not isinstance(testo, str):
        return testo
    testo = unescape(testo)
    testo = re.sub(r'https?://\S+|www\.\S+', '', testo)
    testo = re.sub(r'[^a-zA-Z0-9\s' + re.escape(string.punctuation) + ']', '', testo)
    testo = re.sub(r'\s+', ' ', testo)
    testo = re.sub(r'[\n\t]', ' ', testo)
    testo = re.sub(punct_pattern, ' ', testo)
    testo = re.sub(r'[0-9\.]+', '', testo) 
    testo = testo.strip()
    
    # Filtra le stopwords
    parole = [word for word in testo.split() if word.lower() not in stop_words]
    
    # Ritorna il testo senza stopwords
    return " ".join(parole)

df_emoji["emoji_decoded"] = df_emoji["emoji_decoded"].apply(text_cleaning)

### TRASFORMAZIONE NUMERI IN TESTO (per non perdere informazione)

In [29]:
# traduzione dei numeri in testo
def numbers_to_text(testo):
    if not isinstance(testo, str):
        return testo
    
    def num_to_text(match):
        numero = int(match.group())
        return num2words(numero, lang="en")
    
    return re.sub(r'\b\d+\b', num_to_text, testo)

df_emoji["emoji_decoded"] = df_emoji["emoji_decoded"].apply(numbers_to_text)
df_emoji

,commento,emoji_decoded,emoji_set,emoji_count,parole_maiusc
0,is this a satire subreddit?,satire subreddit,{},0,[]
1,Dayum.,dayum,{},0,[]
2,And they wouldn’t be scrubbing the floors if t...,would scrubbing floors employed,{},0,[]
3,Good point. I mean does Nebraska really exist?...,good point mean nebraska really exist ever met...,{},0,[]
4,or a yuumi/nami/lulu/zilean/ any enhancer play...,yuumi nami lulu zilean enhancer plays team,{},0,[]
...,...,...,...,...,...
5270,Commies are going to commie. These people hate...,commies going commie people hate want die,{},0,[]
5271,Yeah but then the good guys win and rape you t...,yeah good guys win rape death occupy nation in...,{},0,[]
5272,If you're white and use your gun to defend you...,white use gun defend arrested racist get life ...,{},0,[]
5273,I will rape Elon Musk,rape elon musk,{},0,[]


### UNISCO IL DATAFRAME INSIEME A QUELLO PRECEDENTE PER AVERE DATI COMPLETI 

In [30]:
df_copy_2 = pd.concat([df_copy, df_emoji], ignore_index=True, axis=1)

In [31]:
df_copy_2.columns = [
    "commento", "valutazione", "tossicita", "tipo_tossicita", "age_group", "gender", "utente_id",  
    "commento2", "comment_cleaned", "emoji_set", "emoji_count", "parole_maiusc" 
    # rinomino emoji_decoded in comment_cleaned
]
df_copy_2.drop(columns=['commento2'], inplace=True) # Rimuovo la colonna 'commento' duplicata
df_copy_2

,commento,valutazione,tossicita,tipo_tossicita,age_group,gender,utente_id,comment_cleaned,emoji_set,emoji_count,parole_maiusc
0,is this a satire subreddit?,Non toxic,0,0,1,1,1,satire subreddit,{},0,[]
1,Dayum.,Non toxic,0,0,1,1,1,dayum,{},0,[]
2,And they wouldn’t be scrubbing the floors if t...,Non toxic,0,0,1,1,1,would scrubbing floors employed,{},0,[]
3,Good point. I mean does Nebraska really exist?...,Non toxic,0,0,1,1,1,good point mean nebraska really exist ever met...,{},0,[]
4,or a yuumi/nami/lulu/zilean/ any enhancer play...,Non toxic,0,0,1,1,1,yuumi nami lulu zilean enhancer plays team,{},0,[]
...,...,...,...,...,...,...,...,...,...,...,...
5270,Commies are going to commie. These people hate...,Extremely violent,3,4,1,0,211,commies going commie people hate want die,{},0,[]
5271,Yeah but then the good guys win and rape you t...,Extremely violent,3,4,1,0,211,yeah good guys win rape death occupy nation in...,{},0,[]
5272,If you're white and use your gun to defend you...,A little violent,1,4,1,0,211,white use gun defend arrested racist get life ...,{},0,[]
5273,I will rape Elon Musk,A little violent,1,4,1,0,211,rape elon musk,{},0,[]


### POS TAGGING

In [32]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def posTag(text):
    if not isinstance(text, str):
        return []
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return pos_tags

[nltk_data] Downloading package punkt to /Users/User/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/User/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [33]:
df_copy_2["pos_tag"] = df_copy_2["comment_cleaned"].apply(posTag)
df_copy_2

,commento,valutazione,tossicita,tipo_tossicita,age_group,gender,utente_id,comment_cleaned,emoji_set,emoji_count,parole_maiusc,pos_tag
0,is this a satire subreddit?,Non toxic,0,0,1,1,1,satire subreddit,{},0,[],"[(satire, NN), (subreddit, NN)]"
1,Dayum.,Non toxic,0,0,1,1,1,dayum,{},0,[],"[(dayum, NN)]"
2,And they wouldn’t be scrubbing the floors if t...,Non toxic,0,0,1,1,1,would scrubbing floors employed,{},0,[],"[(would, MD), (scrubbing, VBG), (floors, NNS),..."
3,Good point. I mean does Nebraska really exist?...,Non toxic,0,0,1,1,1,good point mean nebraska really exist ever met...,{},0,[],"[(good, JJ), (point, NN), (mean, VBP), (nebras..."
4,or a yuumi/nami/lulu/zilean/ any enhancer play...,Non toxic,0,0,1,1,1,yuumi nami lulu zilean enhancer plays team,{},0,[],"[(yuumi, NN), (nami, NN), (lulu, JJ), (zilean,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5270,Commies are going to commie. These people hate...,Extremely violent,3,4,1,0,211,commies going commie people hate want die,{},0,[],"[(commies, NNS), (going, VBG), (commie, JJ), (..."
5271,Yeah but then the good guys win and rape you t...,Extremely violent,3,4,1,0,211,yeah good guys win rape death occupy nation in...,{},0,[],"[(yeah, RB), (good, JJ), (guys, NNS), (win, VB..."
5272,If you're white and use your gun to defend you...,A little violent,1,4,1,0,211,white use gun defend arrested racist get life ...,{},0,[],"[(white, JJ), (use, NN), (gun, NN), (defend, N..."
5273,I will rape Elon Musk,A little violent,1,4,1,0,211,rape elon musk,{},0,[],"[(rape, NN), (elon, NN), (musk, NN)]"


### NRCLex (misura emozioni all'interno di un testo, analizzando ciascuna parola e la frase nel complesso)

The NRC Emotion Lexicon is a list of English words and their associations with eight basic emotions (anger, fear, anticipation, trust, surprise, sadness, joy, and disgust) and two sentiments (negative and positive). The annotations were manually done by crowdsourcing. 

https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm 

In [34]:
# https://pypi.org/project/NRCLex/
# !pip install NRCLex

from nrclex import NRCLex

# Funzione che restituisce tutti i dati da NRCLex
def nrc_emozioni(text):
    if not isinstance(text, str) or text.strip() == "":
        return {
            "nrc_affect_dict": {},
            "nrc_raw_scores": {},
            "nrc_top_emotions": [],
            "nrc_affect_frequencies": {}
        }

    emotion_obj = NRCLex(text)
    
    return {
        "nrc_affect_dict": emotion_obj.affect_dict,  # mappa parola → emozioni
        "nrc_raw_scores": emotion_obj.raw_emotion_scores,  # conteggi grezzi
        "nrc_top_emotions": emotion_obj.top_emotions,  # emozioni dominanti
        "nrc_affect_frequencies": emotion_obj.affect_frequencies  # frequenze normalizzate
    }

In [35]:
nrc_details = df_copy_2["comment_cleaned"].apply(nrc_emozioni)
nrc_expanded = pd.DataFrame(nrc_details.tolist())

df_copy_2 = pd.concat([df_copy_2, nrc_expanded], axis=1)
df_copy_2

,commento,valutazione,tossicita,tipo_tossicita,age_group,gender,utente_id,comment_cleaned,emoji_set,emoji_count,parole_maiusc,pos_tag,nrc_affect_dict,nrc_raw_scores,nrc_top_emotions,nrc_affect_frequencies
0,is this a satire subreddit?,Non toxic,0,0,1,1,1,satire subreddit,{},0,[],"[(satire, NN), (subreddit, NN)]",{},{},"[(fear, 0.0), (anger, 0.0), (anticip, 0.0), (t...","{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't..."
1,Dayum.,Non toxic,0,0,1,1,1,dayum,{},0,[],"[(dayum, NN)]",{},{},"[(fear, 0.0), (anger, 0.0), (anticip, 0.0), (t...","{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't..."
2,And they wouldn’t be scrubbing the floors if t...,Non toxic,0,0,1,1,1,would scrubbing floors employed,{},0,[],"[(would, MD), (scrubbing, VBG), (floors, NNS),...",{},{},"[(fear, 0.0), (anger, 0.0), (anticip, 0.0), (t...","{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't..."
3,Good point. I mean does Nebraska really exist?...,Non toxic,0,0,1,1,1,good point mean nebraska really exist ever met...,{},0,[],"[(good, JJ), (point, NN), (mean, VBP), (nebras...","{'good': ['anticipation', 'joy', 'positive', '...","{'anticipation': 1, 'joy': 1, 'positive': 1, '...","[(trust, 0.2), (surprise, 0.2), (positive, 0.2...","{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't..."
4,or a yuumi/nami/lulu/zilean/ any enhancer play...,Non toxic,0,0,1,1,1,yuumi nami lulu zilean enhancer plays team,{},0,[],"[(yuumi, NN), (nami, NN), (lulu, JJ), (zilean,...",{'team': ['trust']},{'trust': 1},"[(trust, 1.0)]","{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270,Commies are going to commie. These people hate...,Extremely violent,3,4,1,0,211,commies going commie people hate want die,{},0,[],"[(commies, NNS), (going, VBG), (commie, JJ), (...","{'hate': ['anger', 'disgust', 'fear', 'negativ...","{'anger': 1, 'disgust': 1, 'fear': 2, 'negativ...","[(fear, 0.25), (negative, 0.25), (sadness, 0.25)]","{'fear': 0.25, 'anger': 0.125, 'anticip': 0.0,..."
5271,Yeah but then the good guys win and rape you t...,Extremely violent,3,4,1,0,211,yeah good guys win rape death occupy nation in...,{},0,[],"[(yeah, RB), (good, JJ), (guys, NNS), (win, VB...","{'good': ['anticipation', 'joy', 'positive', '...","{'anticipation': 2, 'joy': 1, 'positive': 2, '...","[(fear, 0.10526315789473684), (anger, 0.105263...","{'fear': 0.10526315789473684, 'anger': 0.10526..."
5272,If you're white and use your gun to defend you...,A little violent,1,4,1,0,211,white use gun defend arrested racist get life ...,{},0,[],"[(white, JJ), (use, NN), (gun, NN), (defend, N...","{'white': ['anticipation', 'joy', 'positive', ...","{'anticipation': 1, 'joy': 1, 'positive': 2, '...","[(fear, 0.25)]","{'fear': 0.25, 'anger': 0.08333333333333333, '..."
5273,I will rape Elon Musk,A little violent,1,4,1,0,211,rape elon musk,{},0,[],"[(rape, NN), (elon, NN), (musk, NN)]","{'rape': ['anger', 'disgust', 'fear', 'negativ...","{'anger': 1, 'disgust': 1, 'fear': 1, 'negativ...","[(fear, 0.2), (anger, 0.2), (negative, 0.2), (...","{'fear': 0.2, 'anger': 0.2, 'anticip': 0.0, 't..."


### POLARITÀ - TEXTBLOB

TextBlob può fare tante cose, ma come funzionalità di libreria siamo interessati solo alla Sentiment Analysis.

The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

La polarità misura se un testo ha un tono positivo, negativo o neutro.

In [36]:
# https://textblob.readthedocs.io/en/dev/quickstart.html

from textblob import TextBlob   

def polarità(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None

df_copy_2["comm_polarita"] = df_copy_2["comment_cleaned"].apply(polarità)
df_copy_2

,commento,valutazione,tossicita,tipo_tossicita,age_group,gender,utente_id,comment_cleaned,emoji_set,emoji_count,parole_maiusc,pos_tag,nrc_affect_dict,nrc_raw_scores,nrc_top_emotions,nrc_affect_frequencies,comm_polarita
0,is this a satire subreddit?,Non toxic,0,0,1,1,1,satire subreddit,{},0,[],"[(satire, NN), (subreddit, NN)]",{},{},"[(fear, 0.0), (anger, 0.0), (anticip, 0.0), (t...","{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...",0.000000
1,Dayum.,Non toxic,0,0,1,1,1,dayum,{},0,[],"[(dayum, NN)]",{},{},"[(fear, 0.0), (anger, 0.0), (anticip, 0.0), (t...","{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...",0.000000
2,And they wouldn’t be scrubbing the floors if t...,Non toxic,0,0,1,1,1,would scrubbing floors employed,{},0,[],"[(would, MD), (scrubbing, VBG), (floors, NNS),...",{},{},"[(fear, 0.0), (anger, 0.0), (anticip, 0.0), (t...","{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...",0.000000
3,Good point. I mean does Nebraska really exist?...,Non toxic,0,0,1,1,1,good point mean nebraska really exist ever met...,{},0,[],"[(good, JJ), (point, NN), (mean, VBP), (nebras...","{'good': ['anticipation', 'joy', 'positive', '...","{'anticipation': 1, 'joy': 1, 'positive': 1, '...","[(trust, 0.2), (surprise, 0.2), (positive, 0.2...","{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...",0.195833
4,or a yuumi/nami/lulu/zilean/ any enhancer play...,Non toxic,0,0,1,1,1,yuumi nami lulu zilean enhancer plays team,{},0,[],"[(yuumi, NN), (nami, NN), (lulu, JJ), (zilean,...",{'team': ['trust']},{'trust': 1},"[(trust, 1.0)]","{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270,Commies are going to commie. These people hate...,Extremely violent,3,4,1,0,211,commies going commie people hate want die,{},0,[],"[(commies, NNS), (going, VBG), (commie, JJ), (...","{'hate': ['anger', 'disgust', 'fear', 'negativ...","{'anger': 1, 'disgust': 1, 'fear': 2, 'negativ...","[(fear, 0.25), (negative, 0.25), (sadness, 0.25)]","{'fear': 0.25, 'anger': 0.125, 'anticip': 0.0,...",-0.800000
5271,Yeah but then the good guys win and rape you t...,Extremely violent,3,4,1,0,211,yeah good guys win rape death occupy nation in...,{},0,[],"[(yeah, RB), (good, JJ), (guys, NNS), (win, VB...","{'good': ['anticipation', 'joy', 'positive', '...","{'anticipation': 2, 'joy': 1, 'positive': 2, '...","[(fear, 0.10526315789473684), (anger, 0.105263...","{'fear': 0.10526315789473684, 'anger': 0.10526...",0.750000
5272,If you're white and use your gun to defend you...,A little violent,1,4,1,0,211,white use gun defend arrested racist get life ...,{},0,[],"[(white, JJ), (use, NN), (gun, NN), (defend, N...","{'white': ['anticipation', 'joy', 'positive', ...","{'anticipation': 1, 'joy': 1, 'positive': 2, '...","[(fear, 0.25)]","{'fear': 0.25, 'anger': 0.08333333333333333, '...",-0.050000
5273,I will rape Elon Musk,A little violent,1,4,1,0,211,rape elon musk,{},0,[],"[(rape, NN), (elon, NN), (musk, NN)]","{'rape': ['anger', 'disgust', 'fear', 'negativ...","{'anger': 1, 'disgust': 1, 'fear': 1, 'negativ...","[(fear, 0.2), (anger, 0.2), (negative, 0.2), (...","{'fear': 0.2, 'anger': 0.2, 'anticip': 0.0, 't...",0.000000


### READABILITY

- <b>Kincaid (Flesch-Kincaid Grade Level)</b>: stima il grado scolastico minimo per comprendere il testo (es. 5.44 ≈ 5ª classe).

- <b>ARI (Automated Readability)</b>:: misura la difficoltà di lettura basata su parole e frasi; restituisce il grado scolastico richiesto.

- <b>Coleman-Liau</b>:: stima il grado scolastico usando lunghezza media delle parole e frasi (usa caratteri, non sillabe).

- F<b>lesch Reading Ease</b>: scala da 0 a 100, dove valori alti = testo più facile da leggere (es. >80 = molto facile).

- <b>Gunning Fog</b>: stima il livello scolastico considerando le parole complesse (più alto = più difficile).

- <b>LIX</b>: indicatore svedese che misura la leggibilità basandosi su lunghezza delle frasi e parole lunghe.

- <b>SMOG</b>: calcola gli anni di istruzione necessari per capire un testo basandosi sulle parole polysyllabic.

- <b>RIX (Readability)</b>: misura simile al LIX, basata su parole lunghe per frase (valori più alti = testo più difficile).

- <b>Dale-Chall</b>: valuta la leggibilità confrontando le parole con una lista di parole “facili” e contando le difficili

In [37]:
# https://pypi.org/project/readability/0.3.2/

#!pip install https://github.com/andreasvc/readability/tarball/master
#!pip install syntok

import readability
import syntok.segmenter as segmenter

def text_readability(text):
    try:
        tokenized = '\n\n'.join(
            '\n'.join(' '.join(token.value for token in sentence)
                      for sentence in paragraph)
            for paragraph in segmenter.analyze(text)
        )
        scores = readability.getmeasures(tokenized, lang='en')
        return scores['readability grades']
    except:
        return {}

In [38]:
readb = df_copy_2["comment_cleaned"].apply(text_readability)
readability_df = pd.DataFrame(readb.tolist())
df_copy_2 = pd.concat([df_copy_2, readability_df], axis=1)
df_copy_2

,commento,valutazione,tossicita,tipo_tossicita,age_group,gender,utente_id,comment_cleaned,emoji_set,emoji_count,...,comm_polarita,Kincaid,ARI,Coleman-Liau,FleschReadingEase,GunningFogIndex,LIX,SMOGIndex,RIX,DaleChallIndex
0,is this a satire subreddit?,Non toxic,0,0,1,1,1,satire subreddit,{},0,...,0.000000,14.690000,14.895000,13.504439,-6.695000,20.800000,52.000000,8.477226,1.0,15.889200
1,Dayum.,Non toxic,0,0,1,1,1,dayum,{},0,...,0.000000,-3.400000,2.620000,-15.988829,121.220000,0.400000,1.000000,3.000000,0.0,15.839600
2,And they wouldn’t be scrubbing the floors if t...,Non toxic,0,0,1,1,1,would scrubbing floors employed,{},0,...,0.000000,3.670000,13.540000,17.961333,75.875000,1.600000,54.000000,3.000000,2.0,12.040900
3,Good point. I mean does Nebraska really exist?...,Non toxic,0,0,1,1,1,good point mean nebraska really exist ever met...,{},0,...,0.195833,9.550000,9.004000,12.991663,44.405000,12.000000,40.000000,10.745967,3.0,5.233000
4,or a yuumi/nami/lulu/zilean/ any enhancer play...,Non toxic,0,0,1,1,1,yuumi nami lulu zilean enhancer plays team,{},0,...,0.000000,9.054286,6.292857,10.211675,42.615714,8.514286,21.285714,8.477226,1.0,13.881486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270,Commies are going to commie. These people hate...,Extremely violent,3,4,1,0,211,commies going commie people hate want die,{},0,...,-0.800000,2.311429,5.620000,9.371697,90.958571,2.800000,21.285714,3.000000,1.0,4.858629
5271,Yeah but then the good guys win and rape you t...,Extremely violent,3,4,1,0,211,yeah good guys win rape death occupy nation in...,{},0,...,0.750000,8.897778,8.190000,12.270926,47.300000,12.488889,20.111111,10.745967,1.0,10.973067
5272,If you're white and use your gun to defend you...,A little violent,1,4,1,0,211,white use gun defend arrested racist get life ...,{},0,...,-0.050000,4.964444,5.050000,8.351025,75.500000,8.044444,20.111111,8.477226,1.0,3.955289
5273,I will rape Elon Musk,A little violent,1,4,1,0,211,rape elon musk,{},0,...,0.000000,1.313333,-1.090000,-2.143827,90.990000,1.200000,3.000000,3.000000,0.0,15.938800


### TOKENIZATION (divido la frase in singole parole, oppure creo bigrammi e trigrammi --> utile per trovare parolacce o parole composte)

In [39]:
import re
import nltk

def tokenization(text):
    if not isinstance(text, str):
        return []
    return re.split(r'\W+', text.lower().strip())

In [40]:
df_copy_2['comment_tokenized'] = df_copy_2['comment_cleaned'].apply(tokenization)
df_copy_2['comment_tok=2'] = df_copy_2['comment_tokenized'].apply(lambda words: list(nltk.ngrams(words, 2)))
df_copy_2['comment_tok=3'] = df_copy_2['comment_tokenized'].apply(lambda words: list(nltk.ngrams(words, 3)))

### LEMMATIZATION (riporto le parole alla forma base del verbo o sostantivo)

In [41]:
from nltk.stem import WordNetLemmatizer

wordlemm = WordNetLemmatizer()

def lemmatization(data, wordlemm):    
    new_words_lemm=[]    
    for row in data:        
        new_words_lemm_tmp=[]        
        for word in row:            
            lemm_word = wordlemm.lemmatize(word)            
            lemm_word = wordlemm.lemmatize(lemm_word, pos='v')            
            new_words_lemm_tmp.append(lemm_word)        
        new_words_lemm.append(new_words_lemm_tmp)
    return new_words_lemm

In [42]:
df_copy_2["comment_lemmatized"] = lemmatization(df_copy_2["comment_tokenized"], wordlemm)

### NRC-VAD (Valence, Arousal and Dominance)

Diversi studi influenti di analisi fattoriale hanno dimostrato che le dimensioni primarie del significato della parola sono la valenza, l'eccitazione e la dominanza (VAD)

- La valenza è la dimensione positiva - negativa o di piacere - dispiacere;
- l'eccitazione è la dimensione passiva eccitata - calma o attiva; e
- il dominio è la dimensione potente - debole o "avere il pieno controllo" - "non avere alcun controllo".

Il lessico NRC Valence, Arousal, and Dominance (VAD) include un elenco di oltre 20.000 parole inglesi e i loro punteggi di valenza, eccitazione e dominanza. Per una determinata parola e una dimensione (V/A/D), i punteggi vanno da 0 (V/A/D più basso) a 1 (V/A/D più alto).

In [43]:
# https://saifmohammad.com/WebPages/nrc-vad.html 

VAD = pd.read_csv('./NRC-VAD/NRC-VAD-Lexicon-v2.1.txt', sep="\t", header=None)
VAD.columns = ["term", "valence", "arousal", "dominance"]
VAD = VAD.dropna(subset=["term"]).copy()
VAD["term"] = VAD["term"].astype(str).str.strip().str.lower()

for c in ["valence", "arousal", "dominance"]:
    VAD[c] = pd.to_numeric(VAD[c], errors="coerce")

VAD = VAD.dropna(subset=["valence", "arousal", "dominance"])
VAD_dict = VAD.set_index("term")[["valence", "arousal", "dominance"]].to_dict("index")

In [44]:
def emotion_VAD(tokens, dim):
    score = []
    seen = set()  # evita conteggi doppi

    for token in tokens:
        # Se è un bigramma (tuple) -> unisce in stringa
        if isinstance(token, tuple):
            token = ' '.join(token)
        token = str(token).strip().lower()
        used_any = False

        # Prova frase intera
        if token in VAD_dict and token not in seen:
            val = VAD_dict[token].get(dim, 0)
            if pd.notna(val):
                score.append(float(val))
                seen.update(token.split())  # segna parole componenti
                used_any = True

        # Fallback su parole singole
        if not used_any:
            for sub_token in token.split():
                sub_token = sub_token.strip().lower()
                if sub_token in VAD_dict and sub_token not in seen:
                    val = VAD_dict[sub_token].get(dim, 0)
                    if pd.notna(val):
                        score.append(float(val))
                        seen.add(sub_token)

    return sum(score) / len(score) if score else 0.0


def analyze_valence(tokens):   
    return emotion_VAD(tokens, 'valence')

def analyze_arousal(tokens):   
    return emotion_VAD(tokens, 'arousal')

def analyze_dominance(tokens): 
    return emotion_VAD(tokens, 'dominance')

In [45]:
df_copy_2["VAD_valence"]   = df_copy_2["comment_tok=2"].apply(analyze_valence)
df_copy_2["VAD_arousal"]   = df_copy_2["comment_tok=2"].apply(analyze_arousal)
df_copy_2["VAD_dominance"] = df_copy_2["comment_tok=2"].apply(analyze_dominance)
df_copy_2

,commento,valutazione,tossicita,tipo_tossicita,age_group,gender,utente_id,comment_cleaned,emoji_set,emoji_count,...,SMOGIndex,RIX,DaleChallIndex,comment_tokenized,comment_tok=2,comment_tok=3,comment_lemmatized,VAD_valence,VAD_arousal,VAD_dominance
0,is this a satire subreddit?,Non toxic,0,0,1,1,1,satire subreddit,{},0,...,8.477226,1.0,15.889200,"[satire, subreddit]","[(satire, subreddit)]",[],"[satire, subreddit]",-0.084000,0.310000,-0.056000
1,Dayum.,Non toxic,0,0,1,1,1,dayum,{},0,...,3.000000,0.0,15.839600,[dayum],[],[],[dayum],0.000000,0.000000,0.000000
2,And they wouldn’t be scrubbing the floors if t...,Non toxic,0,0,1,1,1,would scrubbing floors employed,{},0,...,3.000000,2.0,12.040900,"[would, scrubbing, floors, employed]","[(would, scrubbing), (scrubbing, floors), (flo...","[(would, scrubbing, floors), (scrubbing, floor...","[would, scrub, floor, employ]",-0.102000,0.282500,-0.358500
3,Good point. I mean does Nebraska really exist?...,Non toxic,0,0,1,1,1,good point mean nebraska really exist ever met...,{},0,...,10.745967,3.0,5.233000,"[good, point, mean, nebraska, really, exist, e...","[(good, point), (point, mean), (mean, nebraska...","[(good, point, mean), (point, mean, nebraska),...","[good, point, mean, nebraska, really, exist, e...",0.326000,-0.267333,0.086333
4,or a yuumi/nami/lulu/zilean/ any enhancer play...,Non toxic,0,0,1,1,1,yuumi nami lulu zilean enhancer plays team,{},0,...,8.477226,1.0,13.881486,"[yuumi, nami, lulu, zilean, enhancer, plays, t...","[(yuumi, nami), (nami, lulu), (lulu, zilean), ...","[(yuumi, nami, lulu), (nami, lulu, zilean), (l...","[yuumi, nami, lulu, zilean, enhancer, play, team]",0.687000,0.294500,0.521500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5270,Commies are going to commie. These people hate...,Extremely violent,3,4,1,0,211,commies going commie people hate want die,{},0,...,3.000000,1.0,4.858629,"[commies, going, commie, people, hate, want, die]","[(commies, going), (going, commie), (commie, p...","[(commies, going, commie), (going, commie, peo...","[commie, go, commie, people, hate, want, die]",-0.329714,0.375857,0.045000
5271,Yeah but then the good guys win and rape you t...,Extremely violent,3,4,1,0,211,yeah good guys win rape death occupy nation in...,{},0,...,10.745967,1.0,10.973067,"[yeah, good, guys, win, rape, death, occupy, n...","[(yeah, good), (good, guys), (guys, win), (win...","[(yeah, good, guys), (good, guys, win), (guys,...","[yeah, good, guy, win, rape, death, occupy, na...",0.012750,0.015250,0.121125
5272,If you're white and use your gun to defend you...,A little violent,1,4,1,0,211,white use gun defend arrested racist get life ...,{},0,...,8.477226,1.0,3.955289,"[white, use, gun, defend, arrested, racist, ge...","[(white, use), (use, gun), (gun, defend), (def...","[(white, use, gun), (use, gun, defend), (gun, ...","[white, use, gun, defend, arrest, racist, get,...",-0.115429,0.365714,0.212857
5273,I will rape Elon Musk,A little violent,1,4,1,0,211,rape elon musk,{},0,...,3.000000,0.0,15.938800,"[rape, elon, musk]","[(rape, elon), (elon, musk)]","[(rape, elon, musk)]","[rape, elon, musk]",-0.469000,0.310000,-0.020000


### SALVATAGGIO IN UN FILE CSV

In [46]:
df_copy_2.to_csv("./survey_preprocessed.csv", index=False, encoding='utf-8', sep=';')